In [ ]:
import socket, threading, utils as op, stats as st

rn = 0
receiver = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
receiver.connect(st.ADDR)

# Check if the received frame is corrupted
def isCorrupted(data, crc):
    return op.crc4itu(data) != crc

# Function to handle receiving frames
def recvFrames():
    global rn
    while True:
        try:
            frame = receiver.recv(20).decode()
            recv_n, _, data, crc = op.receiveFrame(frame)
        except ValueError:  # Handles empty string when connection ends
            return
        
        print(f"[RECV] Received frame {recv_n}, data: {data}")
        
        if data == 'q':  # Disconnect signal
            break
        
        if not isCorrupted(data, crc) and recv_n == rn:
            print(f"[CRC SUCCESS] Data CRC: {op.crc4itu(data)}, Frame CRC: {crc}")
            sendAck()
            rn += 1
            print("[ACK SENT] Acknowledgement sent")
        else:
            print(f"[CRC FAILURE] Data CRC: {op.crc4itu(data)}, Frame CRC: {crc}")

# Function to send an acknowledgment
def sendAck():
    ack_frame = op.makeFrame(rn, "11110000")  # Acknowledgment frame
    receiver.send(ack_frame.encode())

# Start receiver thread
receiver_thread = threading.Thread(target=recvFrames)
receiver_thread.start()
receiver_thread.join()

print("[CLOSING] Closing receiver...")
receiver.close()
